In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time

In [2]:
# user = get_upstox_user("sonupal129@gmail.com")
# symbol = Symbol.objects.get(symbol="ashokley")
# # print(user.get_ohlc(user.get_instrument_by_symbol('NSE_EQ', 'bel'), OHLCInterval.Minute_1, datetime.strptime('12/12/2019', '%d/%m/%Y').date(), datetime.strptime('12/12/2019', '%d/%m/%Y').date()))
# user.get_master_contract("NSE_EQ")
# user.get_master_contract("NSE_INDEX")
# # cache_candles_data(user, symbol)
# # get_stock_current_candle("ashokley")

In [3]:
# def function_caller(start_time, end_time, func, *args, **kwargs):
#     """Call function on custom time with interval functionality using celery periodic task"""
#     current_time = datetime.now().time()
#     if start_time < current_time < end_time:
#         func(*args, **kwargs)

In [4]:
# def func(a,b):
#     a= 3
#     b = 4
#     print(a+b)
    
# start = time(9,15)
# end = time(15,30)
    
# function_caller(start,end, func, 3,4)

In [5]:
redis_cache = caches["redis"]
data = redis_cache.get("raju")
my_list = ["Sonu"]
if not data:
    print(1)

1


In [6]:
user = UserProfile.objects.first()
uuser = user.get_upstox_user()

In [23]:
symbol = Symbol.objects.get(symbol="jswsteel")
symbols = Symbol.objects.filter(modified_at__date=datetime.now().date())
from heystox_intraday.intraday_functions_strategy import *

In [53]:
# for symbol in symbols:
# #     print(symbol)
#     print(symbol.has_entry_for_long_short())

symbol.get_stock_movement()

1.0